In [1]:
import pennylane as qml
import tensorflow as tf

## We use here the Barren Plateau circuit from

https://pennylane.ai/qml/demos/tutorial_barren_plateaus.html

and

https://arxiv.org/pdf/1803.11173.pdf

In [119]:
wires = 4
layers = 8
dev1 = qml.device("default.qubit", wires = wires)
dev2 = qml.device("default.qubit", wires = wires)
gate_set = [qml.RX, qml.RY, qml.RZ]

target = np.ones((2**wires,))*(1/np.sqrt(2**wires))
rho_target = np.outer(target ,np.conj(target).T)

def circuit_VQE(params, wires, num_qubits=4, random_gate_sequence=None, layers=1, rho_target=None):
    for i in range(num_qubits):
        qml.RY(np.pi/4, wires=i)
    for j in range(layers):
        for i in range(num_qubits):
            random_gate_sequence[j][i](params[j][i], wires=i)
        for i in range(num_qubits):
            qml.CZ(wires = [i, (i+1)%num_qubits])

def circuit_with_NN(params, num_qubits=4, random_gate_sequence=None, layers=1, rho_target=None):
    for i in range(num_qubits):
        qml.RY(np.pi/4, wires=i)
    for j in range(layers):
        for i in range(num_qubits):
            random_gate_sequence[j][i](params[j][i], wires=i)
        for i in range(num_qubits):
            qml.CZ(wires = [i, (i+1)%num_qubits])
    wirelist = [i for i in range(num_qubits)]
    return [qml.sample(qml.PauliZ(i)) for i in range(num_qubits)]

In [120]:
obs = [qml.PauliZ(i)@qml.PauliZ((i+1)%wires) for i in range(wires)]
coeffs = np.random.rand(len(obs))

gate_sequence = [[np.random.choice(gate_set) for i in range(wires)] for j in range(layers)]

H = qml.vqe.Hamiltonian(coeffs, obs)

cost = qml.VQECost(circuit_VQE, H, dev1, interface="tf")

In [207]:
params1 = tf.Variable(np.random.rand(layers, wires), dtype=tf.float64)
cost(params1, random_gate_sequence=gate_sequence, layers=layers, rho_target=rho_target, num_qubits=wires)

<tf.Tensor: shape=(), dtype=float64, numpy=0.4427791558650135>

In [208]:
opt = tf.keras.optimizers.SGD(learning_rate = 0.1)

for i in range(20):
    with tf.GradientTape() as tape:
        loss = cost(params1, random_gate_sequence=gate_sequence, layers=layers, rho_target=rho_target, num_qubits=wires)

    print(loss.numpy())
    grad = tape.gradient(loss, [params1])
    opt.apply_gradients(zip(grad, [params1]))

0.4427791558650135
0.3663463635701687
0.28514701458761404
0.1973828379872602
0.10080201744768129
-0.007291985714743501
-0.12977256461054
-0.2690135307939723
-0.42580165819913407
-0.5982324640316483
-0.7813968774132899
-0.9684638739135927
-1.1526737129147264
-1.3288279890662484


KeyboardInterrupt: 

## Sample from trained circuit

In [226]:
qcircuit_with_NN = qml.QNode(circuit_with_NN, dev2).to_tf()

samples = qcircuit_with_NN(params1, random_gate_sequence=gate_sequence, layers=layers, rho_target=rho_target, num_qubits=wires)

In [227]:
samples =  tf.transpose(tf.cast(samples, tf.float64, name=None))

In [229]:
adjacency_matrix = np.zeros((wires, wires))
adjacency_matrix[-1][0] = 1.0
ones = np.ones(wires-1)
off_diag = np.diag(ones, k=1)
adjacency_matrix += off_diag
tf.constant(adjacency_matrix, dtype=tf.float64)

<tf.Tensor: shape=(4, 4), dtype=float64, numpy=
array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.]])>

In [230]:
def cost_from_sample(samples, coeffs, adjacency_matrix, index):
    S = tf.transpose(samples)
    B = tf.linalg.matvec(adjacency_matrix, S[:, 0])
    A = samples[:,0]*coeffs
    return tf.tensordot(A,B,1)

In [231]:
def cost_all_samples(samples, coeffs, adjacency_matrix):
    A = tf.multiply(samples, coeffs)
    B = tf.transpose(tf.linalg.matmul(adjacency_matrix, samples, transpose_b=True))
    return tf.reduce_sum(tf.multiply(A,B), 1)

In [232]:
E_list = cost_all_samples(samples, coeffs, adjacency_matrix)
tf.math.reduce_mean(E_list)

<tf.Tensor: shape=(), dtype=float64, numpy=-1.2863729253194292>

In [240]:
tf.keras.backend.set_floatx("float64")

In [241]:
clayer1 = tf.keras.layers.Dense(wires)
clayer2 = tf.keras.layers.Dense(10, activation="relu")
clayer3 = tf.keras.layers.Dense(wires, activation="tanh")
model = tf.keras.models.Sequential([clayer1, clayer2, clayer3])

In [259]:
X = tf.constant(samples)
opt = tf.keras.optimizers.SGD(learning_rate=0.5)

## Train without Keras

In [265]:
def loss_NN(model, x):
    out = model(x)
    E = cost_all_samples(out, coeffs, adjacency_matrix)
    return tf.reduce_mean(E)

In [276]:
def gradient(model, inputs):
    with tf.GradientTape() as tape:
        loss_value = loss_NN(model, inputs)
    return loss_value, tape.gradient(loss_value, model.trainable_variables)

In [277]:
loss(model, X)
gradient(model, X)

(<tf.Tensor: shape=(), dtype=float64, numpy=-0.44720696542477345>,
 [<tf.Tensor: shape=(4, 4), dtype=float64, numpy=
  array([[-0.32343723,  0.07281084, -0.33036798,  0.04938434],
         [ 0.22881632, -0.04446044,  0.27512843, -0.04687224],
         [-0.33334867,  0.1153197 , -0.28966985,  0.08015519],
         [ 0.32665846, -0.1244555 ,  0.30972632, -0.08657577]])>,
  <tf.Tensor: shape=(4,), dtype=float64, numpy=array([-0.27783322,  0.05450332, -0.4305782 ,  0.16331853])>,
  <tf.Tensor: shape=(4, 10), dtype=float64, numpy=
  array([[ 0.05860188, -0.02715119, -0.17546028, -0.05281407, -0.03674424,
           0.05191329,  0.04007243,  0.00520784, -0.09204773, -0.02430526],
         [-0.09705667,  0.03134304,  0.14824387, -0.0342199 , -0.0245758 ,
           0.04417554, -0.00725921, -0.09318289,  0.13281213, -0.05017526],
         [ 0.60854702, -0.16157831, -0.8146754 ,  0.13181464,  0.08595758,
          -0.15043329,  0.05128077,  0.42783169, -0.74246866,  0.17053423],
         [-1.38

In [281]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

for i in range(500):
    loss_value, grads = gradient(model, X)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    print(loss_value.numpy())

-2.5314801175632424
-2.533488911528403
-2.53545918921286
-2.537405643263299
-2.539314565937717
-2.5411952792629466
-2.5430404541667553
-2.5448579464721752
-2.54664285463876
-2.548399563983671
-2.550127475792094
-2.551825755762027
-2.5534994100380706
-2.5551420812789303
-2.5567621200985626
-2.558355271750562
-2.5599222607048517
-2.561467017905028
-2.5629858918998925
-2.5644817627491237
-2.565956056003377
-2.567406501098795
-2.568835142798404
-2.570243678902453
-2.5716309941175597
-2.5729960709674375
-2.5743431794140514
-2.5756706434473733
-2.576978577126371
-2.5782667268049435
-2.579538008495969
-2.580791308613996
-2.5820270167280333
-2.5832448614232297
-2.584445998176826
-2.585631313686449
-2.5868004831992564
-2.58795384564832
-2.5890917306692747
-2.5902141679302644
-2.5913212098705256
-2.5924118757799035
-2.5934883973098475
-2.5945510578754627
-2.595600133320739
-2.596635892150477
-2.5976585957550347
-2.5986684986282267
-2.5996658485784256
-2.6006508869330864
-2.6016238487367818
-2.60

-2.7129786337924853
-2.7130457634398497
-2.7131125945447225
-2.713179129023647
-2.7132453687771165
-2.7133113156897233
-2.7133769716303515
-2.713442338452258
-2.713507417993342
-2.7135722120762047
-2.7136367225083284
-2.7137009510822807
-2.713764899575799
-2.7138285697519566
-2.7138919633593215
-2.7139550821321254
-2.7140179277903265
-2.714080502039849
-2.714142806572643
-2.714204843066883
-2.714266613187081
-2.714328118584188
-2.714389360895809
-2.714450405080248
-2.7145115726382247
-2.714572481451093
-2.71463313308827
-2.7146935291065826
-2.7147536710503726
-2.7148135604516157
-2.7148731988300443
-2.7149325876932835
-2.714991728536939
-2.7150506228447386
-2.7151092720886854
-2.7151676777290685
-2.715225841214701
-2.715283763982942
-2.715341447459878
-2.7153988930603377
-2.7154561021881283
-2.715513076236046
-2.7155698165860063
-2.715626324609172
-2.715682601666042
-2.7157386491065645
-2.715794468270201
-2.715850060486103
-2.7159054270731238
-2.715960569340009
-2.7160154885853856
-2.7

In [282]:
model(X)

<tf.Tensor: shape=(1000, 4), dtype=float64, numpy=
array([[-0.99999974,  0.99999966, -0.99999972,  0.99999995],
       [-0.99999974,  0.99999966, -0.99999972,  0.99999995],
       [-0.99999974,  0.99999966, -0.99999972,  0.99999995],
       ...,
       [ 0.99747462, -0.9968383 ,  0.99639679, -0.99686883],
       [ 0.98977137, -0.98835394,  0.99299741, -0.99299744],
       [-0.99999974,  0.99999966, -0.99999972,  0.99999995]])>